# 5400 Managing Data Project

**Reference**

https://www.alphavantage.co/documentation/#*intelligence*

## Access Data

Import Packages

In [ ]:
# # downloading yahoo finance
# !pip install yfinance

In [5]:
import requests
import pandas as pd
import json
import yfinance as yf

Helper Function

In [6]:
# create a complete api with ticker, topics, time period, and our key
def getSingleRequest(ticker, topic, time_from, time_to, apikey):
  request  = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=" + ticker +  "&topics=" + topic + "&time_from=" + time_from + "&time_to=" + time_to + "&limit=1000" + "&apikey=" + apikey
  return request

#  get the data from the api
def getData(request):
  r = requests.get(request)
  data = r.json()
  return data

# get the top 500 S&P stock name
def get_sp500_list():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    sp500_table = tables[0]["Symbol"].tolist()
    return sp500_table

In [7]:
# 500 list
ticker = get_sp500_list()

# get top 50 list
stocks_info = []
for symbol in ticker:
    stock = yf.Ticker(symbol)
    info = stock.info
    market_cap = info.get('marketCap', 0)
    stocks_info.append((symbol, market_cap))

df = pd.DataFrame(stocks_info, columns=['Symbol', 'MarketCap'])

df_sorted = df.sort_values(by='MarketCap', ascending=False)

top_50_stocks = df_sorted.head(50)

In [8]:
# flatten to list
top_50_stocks_list = top_50_stocks["Symbol"].tolist()

# 15 topics
topics = [
    "blockchain",
    "earnings",
    "ipo",
    "mergers_and_acquisitions",
    "financial_markets",
    "economy_fiscal",
    "economy_monetary",
    "economy_macro",
    "energy_transportation",
    "finance",
    "life_sciences",
    "manufacturing",
    "real_estate",
    "retail_wholesale",
    "technology"
]


In [ ]:
# This is how a api structure

# https://www.alphavantage.co/query?
# function=NEWS_SENTIMENT   NO CAHNGE
# &tickers=COIN,CRYPTO:BTC,FOREX:USD
# &time_from=20220410T0130
# &limit=1000               NO CHANGE
# &apikey=demo
# 150 requrest/min API key: LZQ8M6960ZB021TD


# initiate the data
file_path = '/content/drive/MyDrive/SPRING24/5400 Managing Data/5400 Project/data.json'
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

row_num = len(data)

# finished
# 20220401 - 20220430
# "20220501T0000", "20220531T2359"
# "20220601T0000", "20220630T2359"
# "20220701T0000", "20220731T2359"
# "20220801T0000", "20220831T2359"
# "20220901T0000", "20220930T2359"
# "20221001T0000", "20221031T2359"
# "20221101T0000", "20221130T2359"
# "20221201T0000", "20221231T2359"
# "20230101T0000", "20230131T2359"
# "20230201T0000", "20230228T2359"
# "20230301T0000", "20230331T2359"
# "20230401T0000", "20230430T2359"
# "20230501T0000", "20230531T2359"
# "20230601T0000", "20230630T2359"
# "20230701T0000", "20230731T2359"
# "20230801T0000", "20230831T2359"
# "20230901T0000", "20230930T2359"
# "20231001T0000", "20231031T2359"
# "20231201T0000", "20231231T2359"
# "20240101T0000", "20240131T2359"
# "20240201T0000", "20240229T2359"
# "20240301T0000", "20240331T2359"

# unfinished
# "20220101T0000", "20210131T2359"
# "20220201T0000", "20210228T2359"
# "20220301T0000", "20210331T2359"

In [ ]:
# current time period
period = [
["20220101T0000", "20210131T2359"],
["20220201T0000", "20210228T2359"],
["20220301T0000", "20210331T2359"]
]

In [ ]:
key = "LZQ8M6960ZB021TD"
# access the data with looping
for t1,t2 in period:
  for company in top_50_stocks_list:
      for topic in topics:
          request = getSingleRequest(company, topic,t1, t2, key)
          data_cur = getData(request)

          try:
              row_num += int(data_cur['items'])
              data.extend(data_cur['feed'])
          except KeyError:
              print(f"Error: 'items' key not found for company {company} and topic{topic} during {t1} and {t2}.")
              continue  # Skip to the next iteration of the inner loop
          except ValueError:
              print(f"Error: 'items' key not found for company {company} and topic{topic} during {t1} and {t2}.")
              continue  # Skip to the next iteration of the inner loop

In [ ]:
# print current row number
print(row_num)

843486


In [ ]:
# write data
# LJQ path
file_path = '/content/drive/MyDrive/SPRING24/5400 Managing Data/5400 Project/data.json'


# Write data to JSON file in Google Drive
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

## Transform Data

In [ ]:
# reorganize dataset
def transform_data(data):

    transformed_data = []

    for item in data:
        authors_dict = {author: True for author in item['authors']}
        topics_dict = {topic['topic']: float(topic['relevance_score']) for topic in item['topics']}
        ticker_sentiment_dict = {
            ticker['ticker']: {
                'relevance_score': float(ticker['relevance_score']),
                'ticker_sentiment_score': float(ticker['ticker_sentiment_score']),
                'ticker_sentiment_label': ticker['ticker_sentiment_label']
            } for ticker in item['ticker_sentiment']
        }

        transformed_item = item.copy()
        transformed_item['authors'] = authors_dict
        transformed_item['topics'] = topics_dict
        transformed_item['ticker_sentiment'] = ticker_sentiment_dict

        transformed_data.append(transformed_item)

    return transformed_data

def export_json_data(data, file_path):

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Transform the data
transformed_data = transform_data(data)

# Export the transformed data to a new JSON file
export_json_data(transformed_data, '/content/drive/MyDrive/SPRING24/5400 Managing Data/5400 Project/5400.FP.json')

print("Transformation complete and data exported to '5400.FP.json'.")

In [ ]:
file_path = '/content/drive/MyDrive/SPRING24/5400 Managing Data/5400 Project/5400.FP.json'
with open(file_path, 'r') as json_file:
    tfdata = json.load(json_file)

row_num = len(tfdata)

## Buiding up MongoDB Enviroment

In [ ]:
!pip install pymongo
!pip install dnspython

In [ ]:
from pymongo import MongoClient
import pandas as pd
import json 

client = MongoClient('localhost', 27017)
db = client.apan5400
collection = db['Aricles']

with open('./Articles.json', 'r') as file:
    data = json.load(file)

## MongoDB Queries

如果我没有建好环境 你们可以用这个data

In [12]:
# pipeline = [
#     {"$match": {"overall_sentiment_score": {"$gte": 0}}},
#     {"$project": {"_id": 0, "title": 1}}
# ]

# results = list(collection.aggregate(pipeline))

# for result in results:
#     print(result["title"])